# Compare the Action-Rules package with ActionRulesDiscovery package

## Setting

### Parameters

In [1]:
stable_attributes = ["gender", "SeniorCitizen", "Partner"]
flexible_attributes = ["PhoneService", 
                       "InternetService", 
                       "OnlineSecurity", 
                       "DeviceProtection", 
                       "TechSupport",
                       "StreamingTV"]
target = 'Churn'
min_stable_attributes = 2
min_flexible_attributes = 1 #min 1
min_undesired_support = 20
min_undesired_confidence = 0.6
min_desired_support = 20
min_desired_confidence = 0.6
undesired_state = 'Yes'
desired_state = 'No'

## Data

In [2]:
import pandas as pd

Import data to Pandas DataFrame and increase the table tenfold.

In [3]:
pd.set_option('display.max_columns', None)
data_frame = pd.read_csv("data/telco.csv", sep=";")
data_frame

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## Action-Rules package

In [4]:
from action_rules import ActionRules
# Action Rules Mining
action_rules = ActionRules(
    min_stable_attributes = min_stable_attributes, 
    min_flexible_attributes = min_flexible_attributes, 
    min_undesired_support = min_undesired_support, 
    min_undesired_confidence = min_undesired_confidence, 
    min_desired_support = min_desired_support,
    min_desired_confidence = min_desired_confidence, 
    verbose = False)

Action rules discovery.

In [5]:
def measure_time():
    action_rules.fit(
        data = data_frame, 
        stable_attributes = stable_attributes, 
        flexible_attributes = flexible_attributes, 
        target = target, 
        target_undesired_state = undesired_state,
        target_desired_state = desired_state, 
        use_gpu = False,
        use_sparse_matrix = False,
    )
    print('Number of action rules: ' + str(len(action_rules.get_rules().get_ar_notation())))

Measure time for each parameters combination.

In [6]:
t = %timeit -n1 -r1 -o measure_time()
print('Time: ' + str(t))

Number of action rules: 1113
1.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Time: 1.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# actionRulesDiscovery package

In [7]:
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

actionRulesDiscovery = ActionRulesDiscovery()

In [8]:
def measure_time_2():
    actionRulesDiscovery.load_pandas(data_frame)
    actionRulesDiscovery.fit(stable_attributes = stable_attributes,
         flexible_attributes = flexible_attributes,
         consequent = target,
         conf=min_undesired_confidence * 100,
         supp=-min_desired_support,
         desired_classes = [desired_state],
         is_nan=False,
         is_reduction=True,
         is_strict_flexible=False,
         min_stable_attributes=min_stable_attributes,
         min_flexible_attributes=min_flexible_attributes,
         max_stable_attributes=100,
         max_flexible_attributes=100)
    print('Number of action rules: ' + str(len(actionRulesDiscovery.get_action_rules())))

In [9]:
t = %timeit -n1 -r1 -o measure_time_2()
print('Time: ' + str(t))

/home/jupyter-xsykl04@vse.cz/.local/lib/python3.10/site-packages/actionrules/decisions/decisions.py:81: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self.data = self.data.applymap(str)


Number of action rules: 1113
3.21 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Time: 3.21 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Result

The both packages found the same amount of rules. Action-Rules package is faster.